In [3]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd


## Tape Cleaning

In [3]:
#Read Full_tape.csv
import boto3
from sagemaker import get_execution_role
role = get_execution_role()

#Bucket and Folder
bucket = 'project-b-data'
tape_subfolder = 'proj-b/Tape_csv/Full_tape.csv'


conn = boto3.client('s3')
contents = conn.list_objects(Bucket=bucket, Prefix=tape_subfolder)['Contents']
print(contents)

[{'Key': 'proj-b/Tape_csv/Full_tape.csv', 'LastModified': datetime.datetime(2022, 3, 14, 17, 0, 42, tzinfo=tzlocal()), 'ETag': '"0d7970f8fc9704bd81d31fd093f08d72-103"', 'Size': 857650859, 'StorageClass': 'STANDARD', 'Owner': {'DisplayName': 'awslabsc0w3814274t1644419454', 'ID': '3806969ffdd098e93f906aa25d1354cdd46ea5a8bb1197926acc33a14a7a245d'}}]


In [12]:
%%time
tape = pd.read_csv(f"s3://{bucket}/{tape_subfolder}")


CPU times: user 9.06 s, sys: 2.26 s, total: 11.3 s
Wall time: 46.3 s


The files named Tstyyyy-mm-ddtapes.csv are csv files of the "tape" for
this asset on this exchange: for each transaction it records the time
that the transaction took place, the price per unit for the sale, the
quantity transacted, and details of the buyer and the seller. Note that
times are recorded as number of seconds since the market opened at 8am.
Each daily tape file in this data set is approx 10Mb of data.

In [13]:
tape.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10
0,0,Tst2022-01-04,Exch0Lit,8.192,203,{'pool_id': 'Exch0Lit','type': 'Trade','time': 8.192,'price': 203,'qty': 2,'party1': 'S26','party2': 'B24'}
1,1,Tst2022-01-04,Exch0Lit,8.192,199,{'pool_id': 'Exch0Lit','type': 'Trade','time': 8.192,'price': 199,'qty': 3,'party1': 'S26','party2': 'B08'}
2,2,Tst2022-01-04,Exch0Lit,10.336,196,{'pool_id': 'Exch0Lit','type': 'Trade','time': 10.336,'price': 196,'qty': 3,'party1': 'S06','party2': 'B07'}
3,3,Tst2022-01-04,Exch0Lit,10.672,196,{'pool_id': 'Exch0Lit','type': 'Trade','time': 10.672,'price': 196,'qty': 4,'party1': 'S01','party2': 'B01'}
4,4,Tst2022-01-04,Exch0Lit,11.040,201,{'pool_id': 'Exch0Lit','type': 'Trade','time': 11.04,'price': 201,'qty': 1,'party1': 'S12','party2': 'B31'}


In [14]:
tape.shape

(5230364, 12)

In [15]:
for col in tape:
    print(tape[col].unique())
    

[    0     1     2 ... 64927 64928 64929]
['Tst2022-01-04' 'Tst2022-01-05' 'Tst2022-01-06' 'Tst2022-01-07'
 'Tst2022-01-10' 'Tst2022-01-11' 'Tst2022-01-12' 'Tst2022-01-13'
 'Tst2022-01-14' 'Tst2022-01-17' 'Tst2022-01-18' 'Tst2022-01-19'
 'Tst2022-01-20' 'Tst2022-01-21' 'Tst2022-01-24' 'Tst2022-01-25'
 'Tst2022-01-26' 'Tst2022-01-27' 'Tst2022-01-28' 'Tst2022-01-31'
 'Tst2022-02-01' 'Tst2022-02-02' 'Tst2022-02-03' 'Tst2022-02-04'
 'Tst2022-02-07' 'Tst2022-02-08' 'Tst2022-02-09' 'Tst2022-02-10'
 'Tst2022-02-11' 'Tst2022-02-14' 'Tst2022-02-15' 'Tst2022-02-16'
 'Tst2022-02-17' 'Tst2022-02-18' 'Tst2022-02-21' 'Tst2022-02-22'
 'Tst2022-02-23' 'Tst2022-02-24' 'Tst2022-02-25' 'Tst2022-03-01'
 'Tst2022-03-02' 'Tst2022-03-03' 'Tst2022-03-04' 'Tst2022-03-07'
 'Tst2022-03-08' 'Tst2022-03-09' 'Tst2022-03-10' 'Tst2022-03-11'
 'Tst2022-03-14' 'Tst2022-03-15' 'Tst2022-03-16' 'Tst2022-03-17'
 'Tst2022-03-18' 'Tst2022-03-21' 'Tst2022-03-22' 'Tst2022-03-23'
 'Tst2022-03-24' 'Tst2022-03-25' 'Tst2022-03-28'

In [49]:
new_tape = tape.rename(columns = {"Unnamed: 0":"Number_in_Day","0":"Date",
                       "2":"Time","3":"Price","8":"Quantity",
                      "9":"Party1","10":"Party2"}).drop(['1',"4","5","6","7"],axis = 1)
new_tape

,Number_in_Day,Date,Time,Price,Quantity,Party1,Party2
0,0,Tst2022-01-04,8.192,203,'qty': 2,'party1': 'S26','party2': 'B24'}
1,1,Tst2022-01-04,8.192,199,'qty': 3,'party1': 'S26','party2': 'B08'}
2,2,Tst2022-01-04,10.336,196,'qty': 3,'party1': 'S06','party2': 'B07'}
3,3,Tst2022-01-04,10.672,196,'qty': 4,'party1': 'S01','party2': 'B01'}
4,4,Tst2022-01-04,11.040,201,'qty': 1,'party1': 'S12','party2': 'B31'}
...,...,...,...,...,...,...,...
5230359,64888,Tst2022-04-29,30599.376,134,'qty': 1,'party1': 'S21','party2': 'B07'}
5230360,64889,Tst2022-04-29,30599.520,134,'qty': 1,'party1': 'S03','party2': 'B07'}
5230361,64890,Tst2022-04-29,30599.584,134,'qty': 1,'party1': 'S28','party2': 'B01'}
5230362,64891,Tst2022-04-29,30599.776,134,'qty': 3,'party1': 'S04','party2': 'B01'}


In [50]:
import re
new_tape["Date"] = list(map(lambda x:x[3:],new_tape["Date"]))
new_tape["Quantity"] = list(map(lambda x: float(x[-1]),new_tape["Quantity"]))
new_tape["Party1"] = list(map(lambda x: re.findall(r"S[\d][\d]",x)[0],new_tape["Party1"]))
new_tape["Party2"] = list(map(lambda x: re.findall(r"B[\d][\d]",x)[0],new_tape["Party2"]))
new_tape = new_tape.drop(["Number_in_Day"],axis = 1)
new_tape

,Date,Time,Price,Quantity,Party1,Party2
0,2022-01-04,8.192,203,2.0,S26,B24
1,2022-01-04,8.192,199,3.0,S26,B08
2,2022-01-04,10.336,196,3.0,S06,B07
3,2022-01-04,10.672,196,4.0,S01,B01
4,2022-01-04,11.040,201,1.0,S12,B31
...,...,...,...,...,...,...
5230359,2022-04-29,30599.376,134,1.0,S21,B07
5230360,2022-04-29,30599.520,134,1.0,S03,B07
5230361,2022-04-29,30599.584,134,1.0,S28,B01
5230362,2022-04-29,30599.776,134,3.0,S04,B01


In [51]:
new_tape.dtypes

Date         object
Time        float64
Price         int64
Quantity    float64
Party1       object
Party2       object
dtype: object

In [88]:
new_tape.to_csv(f"Tape_csv/New_Full_tape.csv",index = False)

In [72]:
new_tape = pd.read_csv(f"Tape_csv/New_Full_tape.csv")

In [84]:
new_tape.iloc[13]["Time"]

16.112000000000002

In [81]:
round(8.1919999999998,3)

8.192

In [85]:
new_tape["Time"]  = new_tape["Time"].round(decimals = 3)

In [90]:
new_tape.iloc[13]["Time"]

16.112

In [87]:
new_tape

,Date,Time,Price,Quantity,Party1,Party2
0,2022-01-04,8.192,203,2.0,S26,B24
1,2022-01-04,8.192,199,3.0,S26,B08
2,2022-01-04,10.336,196,3.0,S06,B07
3,2022-01-04,10.672,196,4.0,S01,B01
4,2022-01-04,11.040,201,1.0,S12,B31
...,...,...,...,...,...,...
5230359,2022-04-29,30599.376,134,1.0,S21,B07
5230360,2022-04-29,30599.520,134,1.0,S03,B07
5230361,2022-04-29,30599.584,134,1.0,S28,B01
5230362,2022-04-29,30599.776,134,3.0,S04,B01


In [46]:
#len(pd.read_csv(f"Tape_csv/New_Full_tape.csv")["Number_in_Day"].unique())

64930

In [89]:
#Upload the Data
import boto3
s3 = boto3.Session().resource('s3')
key = 'proj-b/Tape_csv/New_Full_tape.csv' # filepath in s3
doc = "Tape_csv/New_Full_tape.csv"# your local file

s3.Bucket('project-b-data').Object(key).upload_file(doc)

In [55]:
%%time
pd.read_csv('s3://project-b-data/proj-b/Tape_csv/New_Full_tape.csv')

CPU times: user 2.41 s, sys: 600 ms, total: 3.01 s
Wall time: 8.57 s


,Date,Time,Price,Quantity,Party1,Party2
0,2022-01-04,8.192,203,2.0,S26,B24
1,2022-01-04,8.192,199,3.0,S26,B08
2,2022-01-04,10.336,196,3.0,S06,B07
3,2022-01-04,10.672,196,4.0,S01,B01
4,2022-01-04,11.040,201,1.0,S12,B31
...,...,...,...,...,...,...
5230359,2022-04-29,30599.376,134,1.0,S21,B07
5230360,2022-04-29,30599.520,134,1.0,S03,B07
5230361,2022-04-29,30599.584,134,1.0,S28,B01
5230362,2022-04-29,30599.776,134,3.0,S04,B01


## LOB Cleaning

In [4]:
date_list = pd.date_range(start = "2022-01-04",end = "2022-04-29").tolist()
exact_date = [str(i)[0:10] for i in date_list]   #2022-01-04 format"
print(exact_date[:5])

['2022-01-04', '2022-01-05', '2022-01-06', '2022-01-07', '2022-01-08']


In [15]:
LOB = pd.DataFrame()
print(LOB)

Empty DataFrame
Columns: []
Index: []


In [6]:
%%time
pd.read_csv(f's3://project-b-data/proj-b/LOB_csv/2022-01-04-LOB.csv')

CPU times: user 3.41 s, sys: 919 ms, total: 4.33 s
Wall time: 16.5 s


,Unnamed: 0,Time,Bid,Ask,Date
0,0,0.000,NaN,NaN,2022-01-04
1,1,0.016,NaN,NaN,2022-01-04
2,2,0.032,NaN,NaN,2022-01-04
3,3,0.048,NaN,NaN,2022-01-04
4,4,0.064,NaN,NaN,2022-01-04
...,...,...,...,...,...
1912495,1912495,30599.920,"[[220, 4], [211, 1], [175, 4], [170, 5], [161,...","[[236, 2], [411, 5], [436, 1], [659, 1], [725,...",2022-01-04
1912496,1912496,30599.936,"[[220, 4], [211, 1], [175, 4], [170, 5], [161,...","[[236, 2], [411, 5], [436, 1], [659, 1], [725,...",2022-01-04
1912497,1912497,30599.952,"[[220, 4], [211, 1], [175, 4], [170, 5], [161,...","[[236, 2], [411, 5], [436, 1], [659, 1], [725,...",2022-01-04
1912498,1912498,30599.968,"[[220, 4], [211, 1], [175, 4], [170, 5], [161,...","[[236, 2], [411, 5], [436, 1], [611, 1], [725,...",2022-01-04


In [16]:
%%time
total_file_number = 0
for date in exact_date:
    try:
        LOB = LOB.append(pd.read_csv(f's3://project-b-data/proj-b/LOB_csv/{date}-LOB.csv'))
        print(f"{date} Finished, LOB Shape: {LOB.shape}")
        total_file_number += 1
    except FileNotFoundError:
        print(f"No file in {date}.")
    except:
        print(f"Something Wrong in {date}!")

2022-01-04 Finished, LOB Shape: (1912500, 5)
2022-01-05 Finished, LOB Shape: (3825000, 5)
2022-01-06 Finished, LOB Shape: (5737500, 5)
2022-01-07 Finished, LOB Shape: (7650000, 5)
No file in 2022-01-08.
No file in 2022-01-09.
2022-01-10 Finished, LOB Shape: (9562500, 5)
2022-01-11 Finished, LOB Shape: (11475000, 5)
2022-01-12 Finished, LOB Shape: (13387500, 5)
2022-01-13 Finished, LOB Shape: (15300000, 5)
2022-01-14 Finished, LOB Shape: (17212492, 5)
No file in 2022-01-15.
No file in 2022-01-16.
2022-01-17 Finished, LOB Shape: (19124992, 5)
2022-01-18 Finished, LOB Shape: (21037492, 5)
2022-01-19 Finished, LOB Shape: (22949992, 5)
2022-01-20 Finished, LOB Shape: (24862492, 5)
2022-01-21 Finished, LOB Shape: (26774992, 5)
No file in 2022-01-22.
No file in 2022-01-23.
2022-01-24 Finished, LOB Shape: (28687492, 5)
2022-01-25 Finished, LOB Shape: (30599992, 5)
2022-01-26 Finished, LOB Shape: (32512492, 5)
2022-01-27 Finished, LOB Shape: (34424992, 5)
2022-01-28 Finished, LOB Shape: (363374

In [17]:
print(total_file_number)

48


In [18]:
LOB.shape

(91799980, 5)

In [19]:
LOB.head()

,Unnamed: 0,Time,Bid,Ask,Date
0,0,0.000,NaN,NaN,2022-01-04
1,1,0.016,NaN,NaN,2022-01-04
2,2,0.032,NaN,NaN,2022-01-04
3,3,0.048,NaN,NaN,2022-01-04
4,4,0.064,NaN,NaN,2022-01-04


In [20]:
new_LOB = LOB.drop(["Unnamed: 0"],axis = 1)
new_LOB.head()

,Time,Bid,Ask,Date
0,0.000,NaN,NaN,2022-01-04
1,0.016,NaN,NaN,2022-01-04
2,0.032,NaN,NaN,2022-01-04
3,0.048,NaN,NaN,2022-01-04
4,0.064,NaN,NaN,2022-01-04


In [40]:
8.192/0.016

512.0

In [39]:
import ast
8.192/0.016
print(new_LOB.iloc[512]["Time"])
print(ast.literal_eval(new_LOB.iloc[512]["Bid"]))
print(ast.literal_eval(new_LOB.iloc[512]["Ask"]))

8.192
[[203, 2], [199, 3], [195, 4], [187, 5], [181, 1], [163, 1], [89, 5], [55, 3], [18, 3]]
[[207, 4], [211, 4], [214, 2], [220, 5], [226, 4], [254, 5], [504, 5], [595, 2], [765, 3]]


In [42]:
%%time
new_LOB.to_csv(f"s3://project-b-data/proj-b/LOB_csv/Full_LOB/New_Full_LOB.csv",index = False)

CPU times: user 7min 30s, sys: 15.8 s, total: 7min 46s
Wall time: 14min 56s


In [43]:
new_LOB.dtypes

Time    float64
Bid      object
Ask      object
Date     object
dtype: object

In [91]:
#9179980, 4
new_LOB.shape

(91799980, 4)

In [47]:
new_LOB.head()

,Time,Bid,Ask,Date
0,0.000,NaN,NaN,2022-01-04
1,0.016,NaN,NaN,2022-01-04
2,0.032,NaN,NaN,2022-01-04
3,0.048,NaN,NaN,2022-01-04
4,0.064,NaN,NaN,2022-01-04


In [46]:
new_tape.dtypes

Date         object
Time        float64
Price         int64
Quantity    float64
Party1       object
Party2       object
dtype: object

In [48]:
new_tape.head()

,Date,Time,Price,Quantity,Party1,Party2
0,2022-01-04,8.192,203,2.0,S26,B24
1,2022-01-04,8.192,199,3.0,S26,B08
2,2022-01-04,10.336,196,3.0,S06,B07
3,2022-01-04,10.672,196,4.0,S01,B01
4,2022-01-04,11.040,201,1.0,S12,B31


In [55]:
new_tape.shape

(5230364, 6)

In [50]:
new_LOB.iloc[0]["Date"]

'2022-01-04'

In [52]:
new_tape.iloc[0]["Date"] == new_LOB.iloc[0]["Date"]

True

In [54]:
new_tape.iloc[0]["Time"] == new_LOB.iloc[512]["Time"]

True

In [6]:
%%time
new_LOB = pd.read_csv(f"s3://project-b-data/proj-b/LOB_csv/Full_LOB/New_Full_LOB.csv")

CPU times: user 2min 26s, sys: 30 s, total: 2min 56s
Wall time: 19min 20s


In [7]:
new_LOB.shape

(91799980, 4)

In [92]:
new_LOB["Time"] = new_LOB["Time"].round(decimals = 3)

In [15]:
new_tape.loc[new_tape['Date'] == "2022-03-11"]

,Date,Time,Price,Quantity,Party1,Party2
3033657,2022-03-11,2.384,203,2.0,S06,B23
3033658,2022-03-11,5.904,203,1.0,S24,B01
3033659,2022-03-11,5.904,203,3.0,S24,B24
3033660,2022-03-11,6.688,203,1.0,S07,B19
3033661,2022-03-11,6.784,203,3.0,S18,B03
...,...,...,...,...,...,...
3098194,2022-03-11,30598.384,208,2.0,S03,B07
3098195,2022-03-11,30598.640,208,1.0,S18,B07
3098196,2022-03-11,30599.568,210,3.0,S01,B08
3098197,2022-03-11,30599.792,209,1.0,S11,B22


In [14]:
new_LOB.loc[new_LOB['Date'] == "2022-03-11"]

,Time,Bid,Ask,Date


In [16]:
pd.merge(new_tape.loc[new_tape['Date'] == "2022-03-11"],
         new_LOB.loc[new_LOB['Date'] == "2022-03-11"],
         how='left', left_on=['Date','Time'], right_on = ['Date','Time'])

,Date,Time,Price,Quantity,Party1,Party2,Bid,Ask
0,2022-03-11,2.384,203,2.0,S06,B23,NaN,NaN
1,2022-03-11,5.904,203,1.0,S24,B01,NaN,NaN
2,2022-03-11,5.904,203,3.0,S24,B24,NaN,NaN
3,2022-03-11,6.688,203,1.0,S07,B19,NaN,NaN
4,2022-03-11,6.784,203,3.0,S18,B03,NaN,NaN
...,...,...,...,...,...,...,...,...
64537,2022-03-11,30598.384,208,2.0,S03,B07,NaN,NaN
64538,2022-03-11,30598.640,208,1.0,S18,B07,NaN,NaN
64539,2022-03-11,30599.568,210,3.0,S01,B08,NaN,NaN
64540,2022-03-11,30599.792,209,1.0,S11,B22,NaN,NaN


In [93]:
#Can't use merge because of memory error, Should use chunk.
Tape_with_LOB = pd.DataFrame()



In [94]:
%%time
for date in list(new_tape["Date"].unique()):
    new_tape_date = new_tape.loc[new_tape["Date"] == date]
    new_LOB_date = new_LOB.loc[new_LOB["Date"] == date]
    new_df = pd.merge(new_tape_date, new_LOB_date,  how='left', left_on=['Date','Time'], right_on = ['Date','Time'])
    Tape_with_LOB = Tape_with_LOB.append(new_df)
    print(f"{date} Finished.")

2022-01-04 Finished.
2022-01-05 Finished.
2022-01-06 Finished.
2022-01-07 Finished.
2022-01-10 Finished.
2022-01-11 Finished.
2022-01-12 Finished.
2022-01-13 Finished.
2022-01-14 Finished.
2022-01-17 Finished.
2022-01-18 Finished.
2022-01-19 Finished.
2022-01-20 Finished.
2022-01-21 Finished.
2022-01-24 Finished.
2022-01-25 Finished.
2022-01-26 Finished.
2022-01-27 Finished.
2022-01-28 Finished.
2022-01-31 Finished.
2022-02-01 Finished.
2022-02-02 Finished.
2022-02-03 Finished.
2022-02-04 Finished.
2022-02-07 Finished.
2022-02-08 Finished.
2022-02-09 Finished.
2022-02-10 Finished.
2022-02-11 Finished.
2022-02-14 Finished.
2022-02-15 Finished.
2022-02-16 Finished.
2022-02-17 Finished.
2022-02-18 Finished.
2022-02-21 Finished.
2022-02-22 Finished.
2022-02-23 Finished.
2022-02-24 Finished.
2022-02-25 Finished.
2022-03-01 Finished.
2022-03-02 Finished.
2022-03-03 Finished.
2022-03-04 Finished.
2022-03-07 Finished.
2022-03-08 Finished.
2022-03-09 Finished.
2022-03-10 Finished.
2022-03-11 Fi

In [19]:
Tape_with_LOB

,Date,Time,Price,Quantity,Party1,Party2,Bid,Ask
0,2022-01-04,8.192,203,2.0,S26,B24,"[[203, 2], [199, 3], [195, 4], [187, 5], [181,...","[[207, 4], [211, 4], [214, 2], [220, 5], [226,..."
1,2022-01-04,8.192,199,3.0,S26,B08,"[[203, 2], [199, 3], [195, 4], [187, 5], [181,...","[[207, 4], [211, 4], [214, 2], [220, 5], [226,..."
2,2022-01-04,10.336,196,3.0,S06,B07,"[[196, 3], [195, 4], [187, 8], [182, 1], [156,...","[[197, 4], [198, 3], [201, 1], [202, 4], [209,..."
3,2022-01-04,10.672,196,4.0,S01,B01,"[[195, 4], [187, 5], [183, 3], [182, 1], [156,...","[[196, 4], [201, 1], [203, 4], [209, 9], [214,..."
4,2022-01-04,11.040,201,1.0,S12,B31,"[[192, 4], [187, 5], [182, 1], [176, 5], [168,...","[[201, 1], [203, 4], [205, 5], [209, 4], [214,..."
...,...,...,...,...,...,...,...,...
64888,2022-04-29,30599.376,134,1.0,S21,B07,"[[134, 8], [133, 9], [132, 3], [121, 5], [93, ...","[[158, 1], [206, 2], [329, 3], [418, 5], [479,..."
64889,2022-04-29,30599.520,134,1.0,S03,B07,"[[134, 5], [133, 9], [132, 3], [121, 5], [93, ...","[[158, 1], [329, 3], [418, 5], [479, 4], [502,..."
64890,2022-04-29,30599.584,134,1.0,S28,B01,"[[134, 4], [133, 9], [132, 3], [121, 5], [93, ...","[[158, 1], [329, 3], [418, 5], [479, 4], [502,..."
64891,2022-04-29,30599.776,134,3.0,S04,B01,"[[134, 3], [133, 9], [132, 3], [121, 5], [93, ...","[[158, 1], [329, 3], [418, 5], [479, 4], [502,..."


In [65]:
#02-21 duplicate time has NaN!!!!
Tape_with_LOB.loc[Tape_with_LOB["Date"] == "2022-02-21"].head(30)

,Date,Time,Price,Quantity,Party1,Party2,Bid,Ask
0,2022-02-21,3.312,232,1.0,S13,B23,"[[197, 1], [188, 1], [176, 1], [175, 5], [173,...","[[232, 3], [233, 4], [614, 3]]"
1,2022-02-21,4.704,210,4.0,S05,B28,"[[200, 1], [199, 5], [197, 1], [193, 6], [176,...","[[210, 15], [229, 2], [235, 5], [686, 3]]"
2,2022-02-21,7.024,210,4.0,S25,B11,"[[209, 2], [208, 5], [206, 1], [205, 1], [200,...","[[210, 14], [229, 2], [232, 3], [235, 5], [343..."
3,2022-02-21,7.024,210,1.0,S23,B11,"[[209, 2], [208, 5], [206, 1], [205, 1], [200,...","[[210, 14], [229, 2], [232, 3], [235, 5], [343..."
4,2022-02-21,7.248,210,1.0,S23,B01,NaN,NaN
5,2022-02-21,7.328,210,2.0,S00,B14,NaN,NaN
6,2022-02-21,7.888,210,2.0,S00,B10,"[[209, 3], [208, 5], [206, 1], [199, 5], [198,...","[[210, 6], [229, 2], [232, 3], [235, 5], [263,..."
7,2022-02-21,7.904,210,1.0,S01,B31,"[[209, 3], [208, 5], [206, 1], [199, 5], [198,...","[[210, 4], [229, 2], [232, 3], [235, 5], [263,..."
8,2022-02-21,7.904,210,2.0,S11,B31,"[[209, 3], [208, 5], [206, 1], [199, 5], [198,...","[[210, 4], [229, 2], [232, 3], [235, 5], [263,..."
9,2022-02-21,8.048,210,1.0,S11,B16,"[[209, 3], [208, 5], [206, 1], [199, 5], [198,...","[[210, 1], [229, 2], [232, 3], [235, 5], [263,..."


In [71]:
Tape_with_LOB.loc[Tape_with_LOB["Date"] == "2022-02-21"].iloc[19]["Time"]

17.055999999999997

In [66]:
Tape_with_LOB.loc[Tape_with_LOB["Date"] == "2022-01-04"].head(30)

,Date,Time,Price,Quantity,Party1,Party2,Bid,Ask
0,2022-01-04,8.192,203,2.0,S26,B24,"[[203, 2], [199, 3], [195, 4], [187, 5], [181,...","[[207, 4], [211, 4], [214, 2], [220, 5], [226,..."
1,2022-01-04,8.192,199,3.0,S26,B08,"[[203, 2], [199, 3], [195, 4], [187, 5], [181,...","[[207, 4], [211, 4], [214, 2], [220, 5], [226,..."
2,2022-01-04,10.336,196,3.0,S06,B07,"[[196, 3], [195, 4], [187, 8], [182, 1], [156,...","[[197, 4], [198, 3], [201, 1], [202, 4], [209,..."
3,2022-01-04,10.672,196,4.0,S01,B01,"[[195, 4], [187, 5], [183, 3], [182, 1], [156,...","[[196, 4], [201, 1], [203, 4], [209, 9], [214,..."
4,2022-01-04,11.040,201,1.0,S12,B31,"[[192, 4], [187, 5], [182, 1], [176, 5], [168,...","[[201, 1], [203, 4], [205, 5], [209, 4], [214,..."
5,2022-01-04,11.040,203,2.0,S09,B31,"[[192, 4], [187, 5], [182, 1], [176, 5], [168,...","[[201, 1], [203, 4], [205, 5], [209, 4], [214,..."
6,2022-01-04,13.008,197,3.0,S07,B27,"[[197, 3], [196, 2], [191, 7], [188, 5], [187,...","[[200, 6], [203, 5], [205, 2], [206, 2], [207,..."
7,2022-01-04,13.008,196,2.0,S07,B03,"[[197, 3], [196, 2], [191, 7], [188, 5], [187,...","[[200, 6], [203, 5], [205, 2], [206, 2], [207,..."
8,2022-01-04,14.080,193,1.0,S31,B23,"[[193, 4], [192, 4], [189, 5], [187, 1], [186,...","[[196, 2], [197, 1], [198, 6], [199, 5], [202,..."
9,2022-01-04,14.672,195,1.0,S23,B06,"[[194, 4], [193, 3], [189, 5], [183, 3], [170,...","[[195, 1], [196, 2], [197, 5], [198, 1], [199,..."


In [53]:
16.112/0.016

1006.9999999999999

In [56]:
new_LOB.loc[new_LOB["Date"] == "2022-01-04"].iloc[1007]

Time                                               16.112
Bid     [[188, 8], [185, 3], [183, 2], [173, 4], [168,...
Ask     [[198, 1], [201, 2], [202, 7], [207, 4], [209,...
Date                                           2022-01-04
Name: 1007, dtype: object

### New_LOB 16.112

In [55]:
# 16.112
print(type(new_LOB.loc[new_LOB["Date"] == "2022-01-04"].iloc[1007]["Time"]))


<class 'numpy.float64'>


In [58]:
# 16.112
print(type(new_LOB.loc[new_LOB["Date"] == "2022-01-04"].iloc[1007]["Date"]))

<class 'str'>


In [63]:
print(new_LOB.loc[new_LOB["Date"] == "2022-01-04"].iloc[1007]["Time"])
print(new_LOB.loc[new_LOB["Date"] == "2022-01-04"].iloc[1007]["Date"])

16.112
2022-01-04


### Tape_with_LOB 16.112

In [59]:
#16.112
print(type(Tape_with_LOB.loc[Tape_with_LOB["Date"] == "2022-01-04"].iloc[13]["Time"]))
print(type(Tape_with_LOB.loc[Tape_with_LOB["Date"] == "2022-01-04"].iloc[13]["Date"]))

<class 'numpy.float64'>
<class 'str'>


In [64]:
#Can't use "=="  in time, should be round
print(Tape_with_LOB.loc[Tape_with_LOB["Date"] == "2022-01-04"].iloc[13]["Time"])
print(Tape_with_LOB.loc[Tape_with_LOB["Date"] == "2022-01-04"].iloc[13]["Date"])

16.112000000000002
2022-01-04


In [62]:
type(new_LOB.loc[new_LOB["Date"] == "2022-01-04"].iloc[1007]["Time"]) ==\
type(Tape_with_LOB.loc[Tape_with_LOB["Date"] == "2022-01-04"].iloc[0]["Time"])

True

str

In [60]:
print(type(Tape_with_LOB.loc[Tape_with_LOB["Date"] == "2022-01-04"].iloc[0]["Time"]))
print(type(Tape_with_LOB.loc[Tape_with_LOB["Date"] == "2022-01-04"].iloc[0]["Date"]))

<class 'numpy.float64'>
<class 'str'>


In [22]:
Tape_with_LOB.loc[Tape_with_LOB["Date"] == "2022-04-21"]

,Date,Time,Price,Quantity,Party1,Party2,Bid,Ask
0,2022-04-21,7.040,130,2.0,S14,B26,NaN,NaN
1,2022-04-21,7.280,130,3.0,S08,B26,NaN,NaN
2,2022-04-21,7.952,132,3.0,S29,B01,NaN,NaN
3,2022-04-21,7.952,136,1.0,S18,B01,NaN,NaN
4,2022-04-21,9.392,135,1.0,S08,B20,NaN,NaN
...,...,...,...,...,...,...,...,...
64858,2022-04-21,30595.696,112,2.0,S14,B06,NaN,NaN
64859,2022-04-21,30595.696,112,3.0,S20,B06,NaN,NaN
64860,2022-04-21,30595.760,112,1.0,S20,B18,NaN,NaN
64861,2022-04-21,30596.224,113,3.0,S04,B22,NaN,NaN


## After Changing the decimal place

In [95]:
Tape_with_LOB

,Date,Time,Price,Quantity,Party1,Party2,Bid,Ask
0,2022-01-04,8.192,203,2.0,S26,B24,"[[203, 2], [199, 3], [195, 4], [187, 5], [181,...","[[207, 4], [211, 4], [214, 2], [220, 5], [226,..."
1,2022-01-04,8.192,199,3.0,S26,B08,"[[203, 2], [199, 3], [195, 4], [187, 5], [181,...","[[207, 4], [211, 4], [214, 2], [220, 5], [226,..."
2,2022-01-04,10.336,196,3.0,S06,B07,"[[196, 3], [195, 4], [187, 8], [182, 1], [156,...","[[197, 4], [198, 3], [201, 1], [202, 4], [209,..."
3,2022-01-04,10.672,196,4.0,S01,B01,"[[195, 4], [187, 5], [183, 3], [182, 1], [156,...","[[196, 4], [201, 1], [203, 4], [209, 9], [214,..."
4,2022-01-04,11.040,201,1.0,S12,B31,"[[192, 4], [187, 5], [182, 1], [176, 5], [168,...","[[201, 1], [203, 4], [205, 5], [209, 4], [214,..."
...,...,...,...,...,...,...,...,...
64888,2022-04-29,30599.376,134,1.0,S21,B07,"[[134, 8], [133, 9], [132, 3], [121, 5], [93, ...","[[158, 1], [206, 2], [329, 3], [418, 5], [479,..."
64889,2022-04-29,30599.520,134,1.0,S03,B07,"[[134, 5], [133, 9], [132, 3], [121, 5], [93, ...","[[158, 1], [329, 3], [418, 5], [479, 4], [502,..."
64890,2022-04-29,30599.584,134,1.0,S28,B01,"[[134, 4], [133, 9], [132, 3], [121, 5], [93, ...","[[158, 1], [329, 3], [418, 5], [479, 4], [502,..."
64891,2022-04-29,30599.776,134,3.0,S04,B01,"[[134, 3], [133, 9], [132, 3], [121, 5], [93, ...","[[158, 1], [329, 3], [418, 5], [479, 4], [502,..."


In [96]:
Tape_with_LOB.loc[Tape_with_LOB["Date"] == "2022-02-21"].head(30)

,Date,Time,Price,Quantity,Party1,Party2,Bid,Ask
0,2022-02-21,3.312,232,1.0,S13,B23,"[[197, 1], [188, 1], [176, 1], [175, 5], [173,...","[[232, 3], [233, 4], [614, 3]]"
1,2022-02-21,4.704,210,4.0,S05,B28,"[[200, 1], [199, 5], [197, 1], [193, 6], [176,...","[[210, 15], [229, 2], [235, 5], [686, 3]]"
2,2022-02-21,7.024,210,4.0,S25,B11,"[[209, 2], [208, 5], [206, 1], [205, 1], [200,...","[[210, 14], [229, 2], [232, 3], [235, 5], [343..."
3,2022-02-21,7.024,210,1.0,S23,B11,"[[209, 2], [208, 5], [206, 1], [205, 1], [200,...","[[210, 14], [229, 2], [232, 3], [235, 5], [343..."
4,2022-02-21,7.248,210,1.0,S23,B01,"[[209, 2], [208, 5], [206, 1], [205, 1], [200,...","[[210, 9], [229, 2], [232, 3], [235, 5], [343,..."
5,2022-02-21,7.328,210,2.0,S00,B14,"[[209, 2], [208, 5], [206, 1], [200, 5], [198,...","[[210, 8], [229, 2], [232, 3], [235, 5], [343,..."
6,2022-02-21,7.888,210,2.0,S00,B10,"[[209, 3], [208, 5], [206, 1], [199, 5], [198,...","[[210, 6], [229, 2], [232, 3], [235, 5], [263,..."
7,2022-02-21,7.904,210,1.0,S01,B31,"[[209, 3], [208, 5], [206, 1], [199, 5], [198,...","[[210, 4], [229, 2], [232, 3], [235, 5], [263,..."
8,2022-02-21,7.904,210,2.0,S11,B31,"[[209, 3], [208, 5], [206, 1], [199, 5], [198,...","[[210, 4], [229, 2], [232, 3], [235, 5], [263,..."
9,2022-02-21,8.048,210,1.0,S11,B16,"[[209, 3], [208, 5], [206, 1], [199, 5], [198,...","[[210, 1], [229, 2], [232, 3], [235, 5], [263,..."


In [97]:
Tape_with_LOB.loc[Tape_with_LOB["Date"] == "2022-01-04"].head(30)

,Date,Time,Price,Quantity,Party1,Party2,Bid,Ask
0,2022-01-04,8.192,203,2.0,S26,B24,"[[203, 2], [199, 3], [195, 4], [187, 5], [181,...","[[207, 4], [211, 4], [214, 2], [220, 5], [226,..."
1,2022-01-04,8.192,199,3.0,S26,B08,"[[203, 2], [199, 3], [195, 4], [187, 5], [181,...","[[207, 4], [211, 4], [214, 2], [220, 5], [226,..."
2,2022-01-04,10.336,196,3.0,S06,B07,"[[196, 3], [195, 4], [187, 8], [182, 1], [156,...","[[197, 4], [198, 3], [201, 1], [202, 4], [209,..."
3,2022-01-04,10.672,196,4.0,S01,B01,"[[195, 4], [187, 5], [183, 3], [182, 1], [156,...","[[196, 4], [201, 1], [203, 4], [209, 9], [214,..."
4,2022-01-04,11.040,201,1.0,S12,B31,"[[192, 4], [187, 5], [182, 1], [176, 5], [168,...","[[201, 1], [203, 4], [205, 5], [209, 4], [214,..."
5,2022-01-04,11.040,203,2.0,S09,B31,"[[192, 4], [187, 5], [182, 1], [176, 5], [168,...","[[201, 1], [203, 4], [205, 5], [209, 4], [214,..."
6,2022-01-04,13.008,197,3.0,S07,B27,"[[197, 3], [196, 2], [191, 7], [188, 5], [187,...","[[200, 6], [203, 5], [205, 2], [206, 2], [207,..."
7,2022-01-04,13.008,196,2.0,S07,B03,"[[197, 3], [196, 2], [191, 7], [188, 5], [187,...","[[200, 6], [203, 5], [205, 2], [206, 2], [207,..."
8,2022-01-04,14.080,193,1.0,S31,B23,"[[193, 4], [192, 4], [189, 5], [187, 1], [186,...","[[196, 2], [197, 1], [198, 6], [199, 5], [202,..."
9,2022-01-04,14.672,195,1.0,S23,B06,"[[194, 4], [193, 3], [189, 5], [183, 3], [170,...","[[195, 1], [196, 2], [197, 5], [198, 1], [199,..."


In [98]:
Tape_with_LOB.to_csv(f"Tape_csv/Tape_with_LOB.csv",index = False)

In [99]:
import boto3
s3 = boto3.Session().resource('s3')
key = 'proj-b/Tape_with_LOB/Tape_with_LOB.csv' # filepath in s3
doc = "Tape_csv/Tape_with_LOB.csv"# your local file

s3.Bucket('project-b-data').Object(key).upload_file(doc)

# Check NAN number

In [116]:
import numpy as np

In [110]:
Tape_with_LOB.loc[Tape_with_LOB["Date"] == "2022-04-21"]["Bid"].iloc[0]

nan

In [111]:
float("nan")

nan

In [114]:
Tape_with_LOB.loc[Tape_with_LOB["Date"] == "2022-04-21"]["Bid"].iloc[0] == float("nan")

False

In [120]:
np.isnan(Tape_with_LOB.loc[Tape_with_LOB["Date"] == "2022-04-21"]["Bid"].iloc[0]) == np.isnan(float("nan"))

True

In [128]:
np.sum(list(map(lambda x:pd.isna(x),Tape_with_LOB.loc[Tape_with_LOB["Date"] == "2022-04-21"]["Bid"])))

64863

In [129]:
pd.isna(float("nan"))

True

In [131]:
%%time
for date in list(Tape_with_LOB["Date"].unique()):
    nan_number = np.sum(list(map(lambda x:pd.isna(x),Tape_with_LOB.loc[Tape_with_LOB["Date"] == date]["Bid"])))
    print(f"{date} nan Number {nan_number}")
    

2022-01-04 nan Number 0
2022-01-05 nan Number 0
2022-01-06 nan Number 3
2022-01-07 nan Number 0
2022-01-10 nan Number 0
2022-01-11 nan Number 11
2022-01-12 nan Number 11
2022-01-13 nan Number 4
2022-01-14 nan Number 3
2022-01-17 nan Number 3
2022-01-18 nan Number 2
2022-01-19 nan Number 2
2022-01-20 nan Number 2
2022-01-21 nan Number 6
2022-01-24 nan Number 0
2022-01-25 nan Number 4
2022-01-26 nan Number 6
2022-01-27 nan Number 2
2022-01-28 nan Number 1
2022-01-31 nan Number 0
2022-02-01 nan Number 16
2022-02-02 nan Number 3
2022-02-03 nan Number 2
2022-02-04 nan Number 0
2022-02-07 nan Number 0
2022-02-08 nan Number 0
2022-02-09 nan Number 0
2022-02-10 nan Number 1
2022-02-11 nan Number 1
2022-02-14 nan Number 0
2022-02-15 nan Number 1
2022-02-16 nan Number 1
2022-02-17 nan Number 0
2022-02-18 nan Number 4
2022-02-21 nan Number 0
2022-02-22 nan Number 6
2022-02-23 nan Number 3
2022-02-24 nan Number 1
2022-02-25 nan Number 5
2022-03-01 nan Number 9
2022-03-02 nan Number 0
2022-03-03 na